In [ ]:
# | default_exp server

In [ ]:
# | export

import importlib
import sys
import asyncio
from typing import *
from contextlib import contextmanager
from pathlib import Path
import threading
import signal

import multiprocessing

from fastkafka.application import FastKafka
from fastkafka.testing import change_dir

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory
from time import sleep

import nbformat
from nbconvert import PythonExporter

In [ ]:
def generate_app_src(out_path: Union[Path, str]) -> None:
    path = Path("099_Test_Service.ipynb")
    if not path.exists():
        path = Path("..") / "099_Test_Service.ipynb"
    if not path.exists():
        raise ValueError(f"Path '{path.resolve()}' does not exists.")

    with open(path, "r") as f:
        notebook = nbformat.reads(f.read(), nbformat.NO_CONVERT)
        exporter = PythonExporter()
        source, _ = exporter.from_notebook_node(notebook)

    with open(out_path, "w") as f:
        f.write(source)

In [ ]:
with TemporaryDirectory() as d:
    generate_app_src((Path(d) / "main.py"))
    !ls -al {d}
    !cat {d}/main.py | grep @kafka_app

total 20
drwx------ 2 tvrtko tvrtko  4096 Jan 30 16:47 .
drwxrwxrwt 1 root   root    4096 Jan 30 16:47 ..
-rw-rw-r-- 1 tvrtko tvrtko 10829 Jan 30 16:47 main.py
    @kafka_app.consumes()  # type: ignore
    @kafka_app.consumes()  # type: ignore
    @kafka_app.produces()  # type: ignore
    @kafka_app.produces()  # type: ignore
    @kafka_app.produces()  # type: ignore
    @kafka_app.produces()  # type: ignore


In [ ]:
# | export


class ImportFromStringError(Exception):
    pass


def _import_from_string(import_str: str) -> Any:
    """Imports library from string

    Note:
        copied from https://github.com/encode/uvicorn/blob/master/uvicorn/importer.py

    Args:
        import_str: input string in form 'main:app'

    """
    sys.path.append(".")

    if not isinstance(import_str, str):
        return import_str

    module_str, _, attrs_str = import_str.partition(":")
    if not module_str or not attrs_str:
        message = (
            'Import string "{import_str}" must be in format "<module>:<attribute>".'
        )
        typer.secho(f"{message}", err=True, fg=typer.colors.RED)
        raise ImportFromStringError(message.format(import_str=import_str))

    try:
        # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
        module = importlib.import_module(module_str)
    except ImportError as exc:
        if exc.name != module_str:
            raise exc from None
        message = 'Could not import module "{module_str}".'
        raise ImportFromStringError(message.format(module_str=module_str))

    instance = module
    try:
        for attr_str in attrs_str.split("."):
            instance = getattr(instance, attr_str)
    except AttributeError:
        message = 'Attribute "{attrs_str}" not found in module "{module_str}".'
        raise ImportFromStringError(
            message.format(attrs_str=attrs_str, module_str=module_str)
        )

    return instance

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with change_dir(d):
        kafka_app = _import_from_string(f"{src_path.stem}:kafka_app")
        assert isinstance(kafka_app, FastKafka)

[INFO] main: check


In [ ]:
# | export

class FastKafkaServer():
    
    def __init__(self, app: FastKafka):
        self.app = app
        self.should_exit = False
    
    
    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

        await self.app.startup()
        await self._main_loop()
        await self.app.shutdown()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            # Signals can only be listened to from the main thread.
            return

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )
        
        def handle_exit(sig: int) -> None:
            self.should_exit = True
        
        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)
            
            
    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with change_dir(d):
        import_str = f"{src_path.stem}:kafka_app"
        app = _import_from_string(import_str)
        # Patch _main_loop to last x seconds, patch fastkafka 
        # startup and shutdown and assert startup and shutdown called

In [ ]:
#| export

@contextmanager
def run_fastkafka_server(app: FastKafka) -> Generator[None, None, None]:
    app = app
    
    def run(app=app):
        server = FastKafkaServer(app=app)
        server.run()
        
    p = multiprocessing.Process(target=run)
    try:
        p.start()
        yield
    except Exception as e:
        print(f"Exception raised {e=}")
    finally:
        p.terminate()
        p.join()
        p.close()


In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with change_dir(d):
        import_str = f"{src_path.stem}:kafka_app"
        app = _import_from_string(import_str)
        with run_fastkafka_server(app), run_fastkafka_server(app):
            sleep(10)

[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmplubdffvk/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmplubdffvk/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmplubdffvk/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka._components.aiokafka_consumer

[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
